<a href="https://colab.research.google.com/github/Habib0308/University_Final_Year_Project/blob/master/Notebooks/Chicago_Data_Preprocessing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objective of this preprocessing is to create encoded dataset for predicting ward**


**Avoid colab clossing**
```
function ClickConnect(){
    document.querySelector("colab-connect-button").click()
    console.log("Clicked on connect button"); 
}
setInterval(ClickConnect,60000)
```

# **Imports**

In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# mounting google drive

In [142]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Importing Dataset**

In [143]:
file_path= "/content/gdrive/My Drive/Final Year Project/Dataset/dataset_analyzed.csv"
data= pd.read_csv(file_path) #, nrows= 10000
del data["date_of_occurrence"]
df= data.copy()

# Changing Data type

In [144]:
df["ward"]= df["ward"].astype(object)
df["beat"]= df["beat"].astype(object)
df["hour"]= df["hour"].astype(object)

In [145]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5


In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232015 entries, 0 to 232014
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   block                  232015 non-null  object 
 1   iucr                   232015 non-null  object 
 2   primary_description    232015 non-null  object 
 3   secondary_description  232015 non-null  object 
 4   location_description   232015 non-null  object 
 5   arrest                 232015 non-null  object 
 6   domestic               232015 non-null  object 
 7   beat                   232015 non-null  object 
 8   ward                   232015 non-null  object 
 9   fbi_cd                 232015 non-null  object 
 10  latitude               232015 non-null  float64
 11  longitude              232015 non-null  float64
 12  month                  232015 non-null  object 
 13  week_day               232015 non-null  object 
 14  hour                   232015 non-nu

# **One hot encoding for features with many variables**
In this i will pick up top most freqquencty occuring labels from each feature and for the rest of the labels if they occur in the data would have their complete row as zero and they will not contain a column of their own


## Counting columns and unique values for each column before encoding

[onehotencoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [147]:
orig_rows, orig_col= df.shape
print(" Rows",orig_rows,"\n","Columns",orig_col)

 Rows 232015 
 Columns 15


In [148]:
for column in df.columns:
    print(column, ": ", df[column].nunique())

block :  26988
iucr :  314
primary_description :  32
secondary_description :  427
location_description :  162
arrest :  2
domestic :  2
beat :  274
ward :  50
fbi_cd :  26
latitude :  115738
longitude :  115721
month :  12
week_day :  7
hour :  24


In [149]:
df.columns

Index(['block', 'iucr', 'primary_description', 'secondary_description',
       'location_description', 'arrest', 'domestic', 'beat', 'ward', 'fbi_cd',
       'latitude', 'longitude', 'month', 'week_day', 'hour'],
      dtype='object')

# **Data For Predicting Ward**

# **Which values to pick?**

**Top Values :**

* top 10 **primary_description**
 * top 10 **secondary_description** for each primary description
 * top 10 **IUDR** codes for each primary description
* top 10 **blocks** from each and all wards
* top 10 **beats**
* top 30 **location** descriptions
* top 10 **FBI code**

**Others**

* arrest :  2
* domestic :  2
* latitude :  115738
* longitude :  115721
* month :  12
* week_day :  7
* hour :  24
 * dividing into 4 times



# **Encoder Function**

This function accepts:
* **column_name**: The column to encode
* **top_values**: List of labels to encode

In [150]:
def encoder_function(column_name, top_values):
    """
    This function accepts:
    column_name: The column to encode
    top_values: List of values to encode
    """
    for category in top_values:
        new_col= column_name + "_"+ category
        df[new_col]= np.where(df[column_name] == category, 1, 0)
    df.head(3)

# **Getting internal top 10 values function**

In [151]:
def list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count):
    """
    This function accepts:
    column_name: The name of the outer level column
    outer_top: (List) top values of outer column
    no_of_top_topick: Number of top values to pick from inner column
    Column_to_count: Name of column from which you want to pick top values of
    """
    top_values=[]
    for value in outer_top:
        filt= (df[column_name] == value )
        top_values.extend([x for x in df[filt][Column_to_count].value_counts(ascending= False).head(no_of_top_topick).index])
    return top_values

# **Top Labels and their encoding**

## **Top 10 primary description**

In [152]:
top_prim_des= [x for x in df["primary_description"].value_counts(ascending= False).head(10).index]
top_values = top_prim_des.copy()
top_values

['THEFT',
 'BATTERY',
 'CRIMINAL DAMAGE',
 'ASSAULT',
 'DECEPTIVE PRACTICE',
 'OTHER OFFENSE',
 'NARCOTICS',
 'BURGLARY',
 'MOTOR VEHICLE THEFT',
 'ROBBERY']

### **Encoding Top Primary Description values**

In [153]:
column_name= "primary_description"
encoder_function(column_name, top_values)

In [154]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0


## **Top Secondary crimes for the top primary description**

In [155]:
# to pass
column_name= "primary_description" # The name of the outer level column
outer_top= top_prim_des # Top values of outer column
no_of_top_topick =10 #these are how many number of values to pick for the inner top values
Column_to_count= "secondary_description" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))

print(top_values)
print(len(set(top_values)))

['POSS: CANNABIS MORE THAN 30GMS', 'TO VEHICLE', 'HARASSMENT BY ELECTRONIC MEANS', 'FOUND SUSPECT NARCOTICS', 'VIOLATE ORDER OF PROTECTION', 'PURSE-SNATCHING', 'THEFT / RECOVERY - AUTOMOBILE', 'AGGRAVATED: OTHER DANG WEAPON', 'OTHER VEHICLE OFFENSE', 'VEHICLE TITLE/REG OFFENSE', 'ATTEMPT FORCIBLE ENTRY', 'TELEPHONE THREAT', 'INSTITUTIONAL VANDALISM', 'ATTEMPT: STRONGARM-NO WEAPON', 'AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY', 'ATTEMPT THEFT', 'GUN OFFENDER: ANNUAL REGISTRATION', 'HOME INVASION', 'STRONGARM - NO WEAPON', 'ARMED - HANDGUN', 'MANU/DEL:CANNABIS OVER 10 GMS', 'DELIVERY CONTAINER THEFT', 'FALSE/STOLEN/ALTERED TRP', 'TO CITY OF CHICAGO PROPERTY', 'UNLAWFUL ENTRY', 'POCKET-PICKING', 'VEHICULAR HIJACKING', 'RETAIL THEFT', 'AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON', 'DOMESTIC BATTERY SIMPLE', 'ARMED: HANDGUN', 'AGGRAVATED VEHICULAR HIJACKING', 'FINANCIAL IDENTITY THEFT OVER $ 300', 'TO PROPERTY', 'FORGERY', 'OTHER CRIME AGAINST PERSON', 'FORCIBLE ENTRY', 

#### **Encoding Top secondary crimes**

In [156]:
column_name= "secondary_description"
encoder_function(column_name, top_values)

In [157]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,secondary_description_$500 AND UNDER,secondary_description_AGGRAVATED: HANDGUN,"secondary_description_TRUCK, BUS, MOTOR HOME","secondary_description_CYCLE, SCOOTER, BIKE NO VIN","secondary_description_PROTECTED EMPLOYEE - HANDS, FISTS, FEET, NO / MINOR INJURY",secondary_description_AGGRAVATED:KNIFE/CUTTING INSTR,secondary_description_TO FIRE FIGHT.APP.EQUIP,secondary_description_FROM COIN-OP MACHINE/DEVICE,secondary_description_THEFT OF LABOR/SERVICES,secondary_description_MANU/DELIVER: HEROIN (WHITE),secondary_description_AGGRAVATED - OTHER DANGEROUS WEAPON,secondary_description_ARMED: OTHER DANGEROUS WEAPON,secondary_description_POSS: SYNTHETIC DRUGS,secondary_description_CRIMINAL DEFACEMENT,secondary_description_PRO EMP HANDS NO/MIN INJURY,secondary_description_POSS: CANNABIS 30GMS OR LESS,secondary_description_POSS: HEROIN(WHITE),secondary_description_OVER $500,secondary_description_CREDIT CARD FRAUD,secondary_description_FROM BUILDING,secondary_description_POSS: CRACK,secondary_description_COUNTERFEIT CHECK,secondary_description_ATTEMPT - AUTOMOBILE,secondary_description_AGGRAVATED,secondary_description_THEFT/RECOVERY: AUTOMOBILE,"secondary_description_CYCLE, SCOOTER, BIKE WITH VIN",secondary_description_FROM COIN-OPERATED MACHINE OR DEVICE,secondary_description_HARASSMENT BY TELEPHONE,secondary_description_TO STATE SUPPORTED PROPERTY,secondary_description_FRAUD OR CONFIDENCE GAME,secondary_description_AGGRAVATED - HANDGUN,secondary_description_AUTOMOBILE,secondary_description_AGGRAVATED - KNIFE / CUTTING INSTRUMENT,secondary_description_ILLEGAL USE CASH CARD,secondary_description_ATTEMPT - FINANCIAL IDENTITY THEFT,secondary_description_ARMED:KNIFE/CUTTING INSTRUMENT,secondary_description_AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST,secondary_description_STRONG ARM - NO WEAPON,secondary_description_FINANCIAL IDENTITY THEFT $300 AND UNDER,"secondary_description_THEFT/RECOVERY: TRUCK,BUS,MHOME"
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top iucr codes for the top primary description**

In [158]:
# to pass
column_name= "primary_description" # The name of the outer level column
outer_top= top_prim_des # Top values of outer column
no_of_top_topick =10 #these are how many number of values to pick for the inner top values
Column_to_count= "iucr" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))

In [159]:
print(top_values)
print(len(set(top_values)))

['0420', '0880', '033A', '0313', '1811', '1812', '1822', '0918', '1370', '0454', '1305', '0497', '0530', '0520', '502R', '0325', '1150', '5111', '0486', '0430', '1154', '0920', '0895', '0925', '502P', '0850', '1122', '0496', '0870', '0326', '0650', '0865', '0553', '0890', '0935', '0860', '0545', '1320', '2024', '0340', '051B', '4387', '2027', '1156', '1152', '0557', '051A', '0498', '0820', '1345', '1121', '2826', '1310', '0910', '0610', '0937', '0320', '2014', '0312', '0810', '0917', '0330', '0630', '0558', '1340', '1130', '031A', '1375', '2022', '0560', '0460', '0620', '5002', '0927', '1153', '0930', '0915', '0484', '5000', '2028', '1120', '0334', '0554', '2093', '1210', '4650', '2820', '2092', '2825', '041A']
90


#### **Encoding Top IUCR codes**

In [160]:
column_name= "iucr"
encoder_function(column_name, top_values)

In [161]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,iucr_1121,iucr_2826,iucr_1310,iucr_0910,iucr_0610,iucr_0937,iucr_0320,iucr_2014,iucr_0312,iucr_0810,iucr_0917,iucr_0330,iucr_0630,iucr_0558,iucr_1340,iucr_1130,iucr_031A,iucr_1375,iucr_2022,iucr_0560,iucr_0460,iucr_0620,iucr_5002,iucr_0927,iucr_1153,iucr_0930,iucr_0915,iucr_0484,iucr_5000,iucr_2028,iucr_1120,iucr_0334,iucr_0554,iucr_2093,iucr_1210,iucr_4650,iucr_2820,iucr_2092,iucr_2825,iucr_041A
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top Blocks from all wards**

In [162]:
# Getting all blocks
wards= list(df["ward"].unique())
print(wards)

[36.0, 45.0, 28.0, 21.0, 42.0, 39.0, 1.0, 17.0, 8.0, 41.0, 27.0, 32.0, 9.0, 7.0, 48.0, 24.0, 25.0, 22.0, 47.0, 29.0, 26.0, 31.0, 40.0, 49.0, 10.0, 38.0, 33.0, 4.0, 20.0, 18.0, 44.0, 50.0, 5.0, 3.0, 14.0, 37.0, 43.0, 6.0, 34.0, 12.0, 2.0, 16.0, 35.0, 46.0, 15.0, 23.0, 19.0, 11.0, 30.0, 13.0]


In [163]:
# Getting list of blocks that occur in top wards
# to pass
column_name= "ward" # The name of the outer level column
outer_top= wards # Top values of outer column
no_of_top_topick =10 #these are how many number of values to pick for the inner top values
Column_to_count= "block" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))
print(top_values)

['003XX E RANDOLPH ST', '089XX S COMMERCIAL AVE', '016XX W HOWARD ST', '080XX S WESTERN AVE', '010XX N FRANCISCO AVE', '001XX N CENTRAL AVE', '035XX N BROADWAY', '048XX S WOOD ST', '005XX W TAYLOR ST', '005XX N MICHIGAN AVE', '076XX S CICERO AVE', '016XX N WELLS ST', '040XX W LAWRENCE AVE', '046XX W DIVERSEY AVE', '026XX N NARRAGANSETT AVE', '011XX W 63RD ST', '028XX W CERMAK RD', '035XX S RHODES AVE', '004XX E 79TH ST', '087XX S COTTAGE GROVE AVE', '056XX W MADISON ST', '020XX W BALMORAL AVE', '017XX E 67TH ST', '0000X E RANDOLPH ST', '056XX W FULLERTON AVE', '132XX S LANGLEY AVE', '037XX N BROADWAY', '061XX S ASHLAND AVE', '082XX S DREXEL AVE', '003XX W 64TH ST', '034XX W FULLERTON AVE', '094XX S ASHLAND AVE', '065XX S PULASKI RD', '0000X W 95TH ST', '006XX E 131ST ST', '035XX W ARMITAGE AVE', '007XX E 79TH ST', '011XX S CANAL ST', '020XX N HALSTED ST', '064XX N SHERIDAN RD', '013XX S CANAL ST', '060XX W 63RD ST', '024XX W CHICAGO AVE', '035XX S MICHIGAN AVE', '076XX S HALSTED ST', '

### **Encoding Block from wards**

In [164]:
column_name= "block"
encoder_function(column_name, top_values)

In [165]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,block_040XX W DIVERSEY AVE,block_039XX W VAN BUREN ST,block_065XX S DR MARTIN LUTHER KING JR DR,block_027XX N CLYBOURN AVE,block_017XX W PRYOR AVE,block_011XX S CLARK ST,block_040XX W GRAND AVE,block_002XX W 47TH ST,block_020XX N MILWAUKEE AVE,block_025XX N CLARK ST,block_035XX W LAWRENCE AVE,block_039XX W 26TH ST,block_039XX W MADISON ST,block_091XX S COMMERCIAL AVE,block_102XX W ZEMKE RD,block_013XX W 76TH ST,block_040XX N ASHLAND AVE,block_008XX N MICHIGAN AVE,block_054XX N EAST RIVER RD,block_054XX W MADISON ST,block_003XX W CERMAK RD,block_047XX N KIMBALL AVE,block_035XX N CLARK ST,block_063XX S HONORE ST,block_021XX W PETERSON AVE,block_047XX S WESTERN AVE,block_044XX N FRANCISCO AVE,block_072XX S ROCKWELL ST,block_0000X N STATE ST,block_051XX S PULASKI RD,block_024XX W 63RD ST,block_062XX N WESTERN AVE,block_075XX N WESTERN AVE,block_039XX W GLADYS AVE,block_040XX E 134TH ST,block_075XX S ASHLAND AVE,block_024XX S SAWYER AVE,block_036XX S ARCHER AVE,block_025XX N KEDZIE BLVD,block_034XX N WESTERN AVE
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top beats from top wards**

In [166]:
# Getting list of blocks that occur in top wards
# to pass
column_name= "ward" # The name of the outer level column
outer_top= wards # Top values of outer column
no_of_top_topick =1 #these are how many number of values to pick for the inner top values
Column_to_count= "beat" # Name of column from which you want to pick top values of

top_values= list(set(list_inner_top_values(column_name, outer_top, no_of_top_topick, Column_to_count)))
print(top_values)

[2433, 131, 1924, 1033, 524, 1911, 1424, 1935, 913, 533, 1432, 922, 923, 2212, 421, 1833, 1834, 813, 1712, 432, 823, 312, 1723, 1724, 835, 1731, 713, 331, 2512, 1623, 1113, 2521, 2011, 1631, 1121, 2533, 2535, 2412, 621, 622, 623, 1522, 1651, 1011, 1013, 2422, 631, 1914, 123, 124]


#### **Encoding Top Beats**

In [167]:
column_name= "beat"
top_values= list(map(str,top_values))
encoder_function(column_name, top_values)

In [168]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,beat_1432,beat_922,beat_923,beat_2212,beat_421,beat_1833,beat_1834,beat_813,beat_1712,beat_432,beat_823,beat_312,beat_1723,beat_1724,beat_835,beat_1731,beat_713,beat_331,beat_2512,beat_1623,beat_1113,beat_2521,beat_2011,beat_1631,beat_1121,beat_2533,beat_2535,beat_2412,beat_621,beat_622,beat_623,beat_1522,beat_1651,beat_1011,beat_1013,beat_2422,beat_631,beat_1914,beat_123,beat_124
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Top Locations**

In [169]:
top_prim_des= [x for x in df["location_description"].value_counts(ascending= False).head(30).index]
top_values = top_prim_des.copy()
top_values

['STREET',
 'RESIDENCE',
 'APARTMENT',
 'SIDEWALK',
 'OTHER',
 'SMALL RETAIL STORE',
 'RESTAURANT',
 'PARKING LOT/GARAGE(NON.RESID.)',
 'ALLEY',
 'DEPARTMENT STORE',
 'VEHICLE NON-COMMERCIAL',
 'GAS STATION',
 'RESIDENCE PORCH/HALLWAY',
 'RESIDENTIAL YARD (FRONT/BACK)',
 'GROCERY FOOD STORE',
 'RESIDENCE-GARAGE',
 'CONVENIENCE STORE',
 'OTHER (SPECIFY)',
 'CTA TRAIN',
 'SCHOOL, PUBLIC, BUILDING',
 'BAR OR TAVERN',
 'COMMERCIAL / BUSINESS OFFICE',
 'PARK PROPERTY',
 'PARKING LOT / GARAGE (NON RESIDENTIAL)',
 'DRUG STORE',
 'RESIDENCE - PORCH / HALLWAY',
 'RESIDENCE - YARD (FRONT / BACK)',
 'CTA BUS',
 'CTA STATION',
 'BANK']

### **Encoding Top Locations**

In [170]:
column_name= "location_description"
encoder_function(column_name, top_values)

In [171]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,beat_623,beat_1522,beat_1651,beat_1011,beat_1013,beat_2422,beat_631,beat_1914,beat_123,beat_124,location_description_STREET,location_description_RESIDENCE,location_description_APARTMENT,location_description_SIDEWALK,location_description_OTHER,location_description_SMALL RETAIL STORE,location_description_RESTAURANT,location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_ALLEY,location_description_DEPARTMENT STORE,location_description_VEHICLE NON-COMMERCIAL,location_description_GAS STATION,location_description_RESIDENCE PORCH/HALLWAY,location_description_RESIDENTIAL YARD (FRONT/BACK),location_description_GROCERY FOOD STORE,location_description_RESIDENCE-GARAGE,location_description_CONVENIENCE STORE,location_description_OTHER (SPECIFY),location_description_CTA TRAIN,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## **Top FBI Code**

In [172]:
top_prim_des= [x for x in df["fbi_cd"].value_counts(ascending= False).head(10).index]
top_values = top_prim_des.copy()
top_values

['06', '08B', '14', '26', '08A', '11', '18', '05', '07', '03']

### **Encoding top FBI codes**

In [173]:
column_name= "fbi_cd"
encoder_function(column_name, top_values)

In [174]:
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,location_description_STREET,location_description_RESIDENCE,location_description_APARTMENT,location_description_SIDEWALK,location_description_OTHER,location_description_SMALL RETAIL STORE,location_description_RESTAURANT,location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_ALLEY,location_description_DEPARTMENT STORE,location_description_VEHICLE NON-COMMERCIAL,location_description_GAS STATION,location_description_RESIDENCE PORCH/HALLWAY,location_description_RESIDENTIAL YARD (FRONT/BACK),location_description_GROCERY FOOD STORE,location_description_RESIDENCE-GARAGE,location_description_CONVENIENCE STORE,location_description_OTHER (SPECIFY),location_description_CTA TRAIN,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


# **Label Encoding of other  columns**

## **Encoding Arrest Column**

In [175]:
column_name= "arrest"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,location_description_APARTMENT,location_description_SIDEWALK,location_description_OTHER,location_description_SMALL RETAIL STORE,location_description_RESTAURANT,location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_ALLEY,location_description_DEPARTMENT STORE,location_description_VEHICLE NON-COMMERCIAL,location_description_GAS STATION,location_description_RESIDENCE PORCH/HALLWAY,location_description_RESIDENTIAL YARD (FRONT/BACK),location_description_GROCERY FOOD STORE,location_description_RESIDENCE-GARAGE,location_description_CONVENIENCE STORE,location_description_OTHER (SPECIFY),location_description_CTA TRAIN,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,arrest_N,arrest_Y
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1


## **Encoding domestic Column**

In [176]:
column_name= "domestic"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,location_description_OTHER,location_description_SMALL RETAIL STORE,location_description_RESTAURANT,location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_ALLEY,location_description_DEPARTMENT STORE,location_description_VEHICLE NON-COMMERCIAL,location_description_GAS STATION,location_description_RESIDENCE PORCH/HALLWAY,location_description_RESIDENTIAL YARD (FRONT/BACK),location_description_GROCERY FOOD STORE,location_description_RESIDENCE-GARAGE,location_description_CONVENIENCE STORE,location_description_OTHER (SPECIFY),location_description_CTA TRAIN,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,arrest_N,arrest_Y,domestic_N,domestic_Y
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0


## **Encoding domestic Column**

In [177]:
column_name= "domestic"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,location_description_OTHER,location_description_SMALL RETAIL STORE,location_description_RESTAURANT,location_description_PARKING LOT/GARAGE(NON.RESID.),location_description_ALLEY,location_description_DEPARTMENT STORE,location_description_VEHICLE NON-COMMERCIAL,location_description_GAS STATION,location_description_RESIDENCE PORCH/HALLWAY,location_description_RESIDENTIAL YARD (FRONT/BACK),location_description_GROCERY FOOD STORE,location_description_RESIDENCE-GARAGE,location_description_CONVENIENCE STORE,location_description_OTHER (SPECIFY),location_description_CTA TRAIN,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,arrest_N,arrest_Y,domestic_N,domestic_Y
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0


## **Encoding month Column**

In [178]:
column_name= "month"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,location_description_CONVENIENCE STORE,location_description_OTHER (SPECIFY),location_description_CTA TRAIN,"location_description_SCHOOL, PUBLIC, BUILDING",location_description_BAR OR TAVERN,location_description_COMMERCIAL / BUSINESS OFFICE,location_description_PARK PROPERTY,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,arrest_N,arrest_Y,domestic_N,domestic_Y,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0


## **Encoding weekday Column**

In [179]:
column_name= "week_day"

labels = list(df[column_name].unique())
encoder_function(column_name, labels)
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,location_description_PARKING LOT / GARAGE (NON RESIDENTIAL),location_description_DRUG STORE,location_description_RESIDENCE - PORCH / HALLWAY,location_description_RESIDENCE - YARD (FRONT / BACK),location_description_CTA BUS,location_description_CTA STATION,location_description_BANK,fbi_cd_06,fbi_cd_08B,fbi_cd_14,fbi_cd_26,fbi_cd_08A,fbi_cd_11,fbi_cd_18,fbi_cd_05,fbi_cd_07,fbi_cd_03,arrest_N,arrest_Y,domestic_N,domestic_Y,month_Jun,month_Jul,month_Aug,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,week_day_Wednesday
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## **Encoding hour Column**

In [180]:
column_name= "hour"

labels = list(df[column_name].unique())

labels= list(map(str,labels))
encoder_function(column_name, labels)
df.head(2)

,block,iucr,primary_description,secondary_description,location_description,arrest,domestic,beat,ward,fbi_cd,latitude,longitude,month,week_day,hour,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,...,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,week_day_Wednesday,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_0,hour_1,hour_2,hour_3,hour_4
0,061XX W GRAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,N,N,2512,36,08B,41.921325,-87.779766,Jun,Thursday,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,043XX N CICERO AVE,0560,ASSAULT,SIMPLE,CTA BUS,Y,N,1624,45,08A,41.960495,-87.747514,Jun,Thursday,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Scaling Values**

## **Scaling longitude Column**

In [181]:
df["latitude"].head(5)

0    41.921325
1    41.960495
2    41.866332
3    41.740986
4    41.893282
Name: latitude, dtype: float64

## **Scaling latitude Column**

In [182]:
df["latitude"].head(5)

0    41.921325
1    41.960495
2    41.866332
3    41.740986
4    41.893282
Name: latitude, dtype: float64

# **Deleting Label Columns**

In [183]:
data.columns

Index(['block', 'iucr', 'primary_description', 'secondary_description',
       'location_description', 'arrest', 'domestic', 'beat', 'ward', 'fbi_cd',
       'latitude', 'longitude', 'month', 'week_day', 'hour'],
      dtype='object')

In [184]:
del df["block"]
del df["iucr"]
del df["primary_description"]
del df["secondary_description"]
del df["location_description"]
del df["arrest"]
del df["domestic"]
del df["beat"]
del df["fbi_cd"]
del df["month"]
del df["week_day"]
del df["hour"]

In [185]:
df.head(2)

,ward,latitude,longitude,primary_description_THEFT,primary_description_BATTERY,primary_description_CRIMINAL DAMAGE,primary_description_ASSAULT,primary_description_DECEPTIVE PRACTICE,primary_description_OTHER OFFENSE,primary_description_NARCOTICS,primary_description_BURGLARY,primary_description_MOTOR VEHICLE THEFT,primary_description_ROBBERY,secondary_description_POSS: CANNABIS MORE THAN 30GMS,secondary_description_TO VEHICLE,secondary_description_HARASSMENT BY ELECTRONIC MEANS,secondary_description_FOUND SUSPECT NARCOTICS,secondary_description_VIOLATE ORDER OF PROTECTION,secondary_description_PURSE-SNATCHING,secondary_description_THEFT / RECOVERY - AUTOMOBILE,secondary_description_AGGRAVATED: OTHER DANG WEAPON,secondary_description_OTHER VEHICLE OFFENSE,secondary_description_VEHICLE TITLE/REG OFFENSE,secondary_description_ATTEMPT FORCIBLE ENTRY,secondary_description_TELEPHONE THREAT,secondary_description_INSTITUTIONAL VANDALISM,secondary_description_ATTEMPT: STRONGARM-NO WEAPON,secondary_description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,secondary_description_ATTEMPT THEFT,secondary_description_GUN OFFENDER: ANNUAL REGISTRATION,secondary_description_HOME INVASION,secondary_description_STRONGARM - NO WEAPON,secondary_description_ARMED - HANDGUN,secondary_description_MANU/DEL:CANNABIS OVER 10 GMS,secondary_description_DELIVERY CONTAINER THEFT,secondary_description_FALSE/STOLEN/ALTERED TRP,secondary_description_TO CITY OF CHICAGO PROPERTY,secondary_description_UNLAWFUL ENTRY,secondary_description_POCKET-PICKING,secondary_description_VEHICULAR HIJACKING,...,month_Sep,month_Oct,month_Nov,month_Dec,month_Jan,month_Feb,month_Mar,month_Apr,month_May,week_day_Thursday,week_day_Friday,week_day_Saturday,week_day_Sunday,week_day_Monday,week_day_Tuesday,week_day_Wednesday,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_0,hour_1,hour_2,hour_3,hour_4
0,36,41.921325,-87.779766,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,45,41.960495,-87.747514,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **Saving Ward data**

In [186]:
df.to_csv("ward_data.csv")